## Simulate with ECLYPSE

This notebook will walk you through the process of setting up and running your first _simulation_ using ECLYPSE, which is made by three main components:

- **Infrastructure**: the set of computing resources on which the application will be deployed.
- **Application**: the software system(s) you want to deploy and simulate.
- **Simulation**: th environment where you can deploy and run your application(s) on the provided infrastructure.

In this first notebook we focus on the **local** simulation, which runs by placing applications on an infrastructure using specific _placement strategies_. The applications and infrastructure are updated during the simulation according to their _update policies_, with updates occurring at each triggered tick event.

To make everything reproducible, we fix a SEED for the simulation. Try to change the SEED and see how the results change!

In [1]:
# Seed for the simulation
SEED = 2

## Build an Infrastructure

The infrastructure represents the computing resources on which the application(s) will be deployed. 
To define an infrastructure, you need to specify its nodes and links, together with the update policy that will be applied to the infrastructure.

First of all, define an update policy that randomly changes the nodes and links resources. 
To do so we just have to define two functions, taking as argument a `networkx.NodeView` and a `networkx.EdgeView`, respectively, and returning the updated set of nodes and edges.

In [2]:
import random as rnd

from networkx.classes.reportviews import (
    EdgeView,
    NodeView,
)


# update edges
def node_random_update(nodes: NodeView):
    for _, resources in nodes.data():
        if rnd.random() < 0.02:
            resources["availability"] = 0
        elif rnd.random() < 0.5 and resources["availability"] == 0:
            resources["availability"] = 1
        else:
            # Randomly update resources with different ranges
            resources["cpu"] = round(max(0, resources["cpu"] * rnd.uniform(0.95, 1.05)))
            resources["gpu"] = round(max(0, resources["gpu"] * rnd.uniform(0.9, 1.1)))
            resources["ram"] = round(max(0, resources["ram"] * rnd.uniform(0.8, 1.2)))
            resources["storage"] = round(
                max(0, resources["storage"] * rnd.uniform(0.9, 1.1))
            )
            resources["availability"] = min(
                1, max(0, resources["availability"] * rnd.uniform(0.995, 1.005))
            )


def edge_random_update(edges: EdgeView):
    for _, _, resources in edges.data():
        # Randomly update resources with different ranges
        resources["latency"] = round(
            max(0, resources["latency"] * rnd.uniform(0.9, 1.1))
        )
        resources["bandwidth"] = round(
            max(0, resources["bandwidth"] * rnd.uniform(0.95, 1.05))
        )

Now you can define the topology, by adding ndoes and links to an extend `networx.Graph`, that is our `eclypse.graph.Infrastructure` class.
The previsouly defined update policies will be parameters of the Infrastructure builder.

In [3]:
from eclypse.graph import Infrastructure, NodeGroup

echo_infra = Infrastructure(
    "EchoInfrastructure",
    node_update_policy=node_random_update,
    edge_update_policy=edge_random_update,
    seed=SEED,
)
echo_infra.add_node_by_group(NodeGroup.CLOUD, "CloudServer")
echo_infra.add_node_by_group(NodeGroup.FAR_EDGE, "EdgeGateway")
echo_infra.add_node_by_group(NodeGroup.IOT, "IoTDevice")
echo_infra.add_node_by_group(NodeGroup.CLOUD, "CloudStorage")
echo_infra.add_node_by_group(NodeGroup.NEAR_EDGE, "EdgeSensor")

echo_infra.add_symmetric_edge("CloudServer", "EdgeGateway", latency=5.0, bandwidth=80.0)
echo_infra.add_symmetric_edge("EdgeGateway", "IoTDevice", latency=8.0, bandwidth=50.0)
echo_infra.add_symmetric_edge(
    "IoTDevice", "CloudStorage", latency=15.0, bandwidth=100.0
)
echo_infra.add_symmetric_edge("CloudStorage", "EdgeSensor", latency=9.0, bandwidth=70.0)

2024-11-25 17:10:22,692	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


## Define the Application

The second step is to define the application that will be placed on the infrastructure. Similarly to the infrastructure, the application is defined by a set of components (services) and edges (interactions), and an update policy that will be applied to the application. In this example we do not change the application requirements, so no update policy is defined.

The application is an instance of another extended graph, specifically the `eclypse.graph.Application` class.

In [4]:
from eclypse.graph import Application, NodeGroup

echo_app = Application("EchoApp")

echo_app.add_node(
    "Gateway",
    cpu=1,
    gpu=0,
    ram=0.5,
    storage=0.5,
    availability=0.9,
    group=NodeGroup.IOT,
    processing_time=0.1,
)

echo_app.add_node(
    "SecurityService",
    cpu=2,
    gpu=0,
    ram=4.0,
    storage=2.0,
    availability=0.8,
    group=NodeGroup.NEAR_EDGE,
    processing_time=2.0,
)

echo_app.add_node(
    "LightingService",
    cpu=1,
    gpu=0,
    ram=2.0,
    storage=5.0,
    availability=0.8,
    group=NodeGroup.IOT,
    processing_time=1.0,
)

echo_app.add_node(
    "ClimateControlService",
    cpu=2,
    gpu=0,
    ram=3.0,
    storage=8.0,
    availability=0.85,
    group=NodeGroup.NEAR_EDGE,
    processing_time=1.5,
)


echo_app.add_node(
    "EntertainmentService",
    cpu=3,
    gpu=1,
    ram=4.0,
    storage=10.0,
    availability=0.9,
    group=NodeGroup.CLOUD,
    processing_time=5.0,
)

echo_app.add_symmetric_edge(
    "Gateway",
    "LightingService",
    latency=100.0,
    bandwidth=20.0,
)

echo_app.add_symmetric_edge(
    "Gateway",
    "ClimateControlService",
    latency=100.0,
    bandwidth=10.0,
)

echo_app.add_symmetric_edge(
    "Gateway",
    "SecurityService",
    latency=50.0,
    bandwidth=5.0,
)

echo_app.add_symmetric_edge(
    "SecurityService",
    "EntertainmentService",
    latency=50.0,
    bandwidth=10.0,
)

## Configure and run the simulation

After you define your application(s) and infrastructure, in order to run the simulation, you need to **configure** your simulation by specifying parameters such as its duration, the report and logging features, and other additional settings. 
To do so, it is necessary to define an instance of the `eclypse.simualation.SimulationConfig` class, specifying all possible parameters in the constructor.

For example, if you want to run a simulation lasting 100 ticks, save the report in `./my-simulation` folder and set a _seed_ for pseudo-randomness, you will have to configure the simulation as follows:

In [5]:
from eclypse.simulation import SimulationConfig
from eclypse.utils import DEFAULT_SIM_PATH

sim_config = SimulationConfig(
    seed=SEED,
    max_ticks=100,
    path="./my-simulation",
    )

Then create an instance of the `eclypse.simulation.Simulation` class, passing the infrastructur and the configuration as arguments:

In [6]:
from eclypse.simulation import Simulation

simulation = Simulation(infrastructure=echo_infra, simulation_config=sim_config)

Now that you have a Simulation object, you can include the application(s) in the simulation by calling the `register` method, passing the application object as an argument, and specifying the **placement strategy** to use. In this case we use one of the off-the-shelf placement strategies provided by ECLYPSE, the `eclypse.placement.strategies.RandomStrategy`:

In [7]:
from eclypse.placement.strategies import RandomStrategy

strategy = RandomStrategy(seed=SEED)

simulation.register(echo_app, placement_strategy=strategy)

Finally we can run the simulation by starting it with the `start` method, then waiting for it to finish with the `wait` method:

In [8]:
simulation.start()
simulation.wait()

17:10:23 | 🌘 ECLYPSE | Simulation - Event Start-0 fired.
17:10:23 | 🌘 ECLYPSE | Simulation - Event Tick-0 fired.
17:10:23 | 🌘 ECLYPSE | Simulation - Event Enact-0 fired.
17:10:23 | 🌘 ECLYPSE | EchoInfrastructure - Node EdgeGateway not respected
17:10:23 | 🌘 ECLYPSE | EchoInfrastructure - Required: {'cpu': 4.0, 'ram': 4.5, 'storage': 10.5, 'gpu': 1.0, 'availability': 0.9, 'processing_time': 0.1, 'group': CLOUD}
17:10:23 | 🌘 ECLYPSE | EchoInfrastructure - Available: {'cpu': 31, 'ram': 66, 'storage': 2244, 'gpu': 25, 'availability': 0.9777402078805711, 'processing_time': 7.906020085560016, 'group': FAR_EDGE}
17:10:23 | 🌘 ECLYPSE | Simulation - Event Tick-1 fired.
17:10:23 | 🌘 ECLYPSE | Simulation - Event Enact-1 fired.
17:10:23 | 🌘 ECLYPSE | EchoInfrastructure - Node IoTDevice not respected
17:10:23 | 🌘 ECLYPSE | EchoInfrastructure - Required: {'cpu': 2.0, 'ram': 3.0, 'storage': 8.0, 'gpu': 0.0, 'availability': 0.85, 'processing_time': 1.5, 'group': NEAR_EDGE}
17:10:23 | 🌘 ECLYPSE | EchoI

Once the simulation is finished, you can access the results by calling the `report` property on the simulation object. This will return a `eclypse.report.Report` object, a friendly interface to select and filter the results of the simulation, that will retrieve pandas dataframes for further analysis.

For instance if you want to get only the metrics related to the application, you can use the `application` method on the report object:

In [ ]:
simulation.report.application()

AAAAA


,timestamp,event_id,n_event,callback_id,application_id,value
0,2024-11-25 17:10:23.082431,enact,1,response_time,EchoApp,inf
1,2024-11-25 17:10:23.089682,enact,2,response_time,EchoApp,inf
2,2024-11-25 17:10:23.095473,enact,3,response_time,EchoApp,inf
3,2024-11-25 17:10:23.102606,enact,4,response_time,EchoApp,inf
4,2024-11-25 17:10:23.109546,enact,5,response_time,EchoApp,inf
...,...,...,...,...,...,...
95,2024-11-25 17:10:23.665646,enact,96,response_time,EchoApp,inf
96,2024-11-25 17:10:23.672976,enact,97,response_time,EchoApp,inf
97,2024-11-25 17:10:23.680004,enact,98,response_time,EchoApp,inf
98,2024-11-25 17:10:23.686890,enact,99,response_time,EchoApp,inf
